In [1]:
import pandas as pd
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

43661


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [18]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
# add_alternate_names = (
#     "INSERT INTO alternative_name "
#     "(name, taxonomy, gene, splice, histone) "
#     "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
# )
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

# Add Singlets, Doublets, Triplets and Quadruplets nodes

In [8]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "H3"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
156,H3,type,Eukaryotes,2759,5.0,None


In [9]:
data_histone = {
    "id": "Singlet",
    "level": 'type',
    "taxonomic_span": 'Archaea, Bacteria, Viruses',
    "taxonomic_span_id": '2157, 2, 10239',
    "description": None,
    "parent": None,
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [10]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Singlet"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
196,Singlet,type,"Archaea, Bacteria, Viruses","2157, 2, 10239",NaN,None


In [11]:
data_histone = {
    "id": "Doublet",
    "level": 'type',
    "taxonomic_span": 'Archaea, Bacteria, Viruses',
    "taxonomic_span_id": '2157, 2, 10239',
    "description": None,
    "parent": None,
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [12]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Doublet"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
75,Doublet,type,"Archaea, Bacteria, Viruses","2157, 2, 10239",NaN,None


In [13]:
data_histone = {
    "id": "Triplet",
    "level": 'type',
    "taxonomic_span": 'Viruses',
    "taxonomic_span_id": '10239',
    "description": None,
    "parent": None,
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [14]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Triplet"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
199,Triplet,type,Viruses,10239,NaN,None


In [15]:
data_histone = {
    "id": "Quadruplet",
    "level": 'type',
    "taxonomic_span": 'Viruses',
    "taxonomic_span_id": '10239',
    "description": None,
    "parent": None,
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [16]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Quadruplet"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
195,Quadruplet,type,Viruses,10239,NaN,None


# Создадим описания

In [19]:
histone_description_summary = "Singlet is one of the histone types present in Archaea, Bacteria and Viruses."
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Singlet'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [20]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Singlet'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Singlet,type,"Archaea, Bacteria, Viruses","2157, 2, 10239",207,None,207,Singlet is one of the histone types present in...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [21]:
histone_description_summary = 'Doublet is one of the histone types present in Archaea, Bacteria and Viruses. Doublets are histone proteins in which two histone-fold domains (structural motifs responsible for DNA binding) are fused into a single gene. These domains are arranged sequentially ("back-to-back") and linked by a flexible peptide connector, allowing the protein to fold into a functional dimer. '
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Doublet'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [22]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Doublet'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Doublet,type,"Archaea, Bacteria, Viruses","2157, 2, 10239",208,None,208,Doublet is one of the histone types present in...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [23]:
histone_description_summary = 'Triplet is one of the histone types present in Viruses. Triplets are histone proteins in which tree histone-fold domains (structural motifs responsible for DNA binding) are fused into a single gene. These domains are arranged sequentially ("back-to-back") and linked by a flexible peptide connector. '
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Triplet'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [24]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Triplet'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Triplet,type,Viruses,10239,209,None,209,Triplet is one of the histone types present in...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [25]:
histone_description_summary = 'Quadruplet is one of the histone types present in Viruses. Quadruplets are histone proteins in which four histone-fold domains (structural motifs responsible for DNA binding) are fused into a single gene. These domains are arranged sequentially ("back-to-back") and linked by a flexible peptide connector. '
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Quadruplet'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [26]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Quadruplet'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Quadruplet,type,Viruses,10239,210,None,210,Quadruplet is one of the histone types present...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Capstones

In [27]:
data_histone = {
    "id": "Capstone",
    "level": 'type',
    "taxonomic_span": 'Archaea, Bacteria, Viruses',
    "taxonomic_span_id": '2157, 2, 10239',
    "description": None,
    "parent": 'Singlet',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [30]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
1,Capstone,type,"Archaea, Bacteria, Viruses","2157, 2, 10239",NaN,Singlet


In [31]:
query = f"UPDATE histone SET level='variant_group' WHERE id='Capstone'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
1,Capstone,variant_group,"Archaea, Bacteria, Viruses","2157, 2, 10239",NaN,Singlet


In [32]:
histone_description_summary = 'Capstones are histone variants found in archaea that hinder stable tetramer formation and further elongation of oligomeric chromatin structures. They are distinguished by their unstable interfaces between dimers, frequently resulting from key amino acid substitutions (e.g., the loss of histidine at the end of α2 in the capstone of Methanobrevibacter smithii), which disrupts interactions between histone complexes [stevens_histone_2023, stevens_histone_2020, henneman_structure_2018]. These histones act as "capstones", thereby limiting DNA compactization and regulating genome accessibility. It is noteworthy that capstones are evolutionarily conserved and have arisen independently in different lineages of archaea, which underscores their significance in chromatin organisation, akin to functional variants of histones in eukaryotes [stevens_histone_2020].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Capstone'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [33]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Capstone'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Capstone,variant_group,"Archaea, Bacteria, Viruses","2157, 2, 10239",211,Singlet,211,Capstones are histone variants found in archae...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add FtF

In [34]:
data_histone = {
    "id": "FtF",
    "level": 'variant_group',
    "taxonomic_span": 'Archaea, Bacteria',
    "taxonomic_span_id": '2157, 2',
    "description": None,
    "parent": 'Singlet',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [35]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "FtF"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
78,FtF,variant_group,"Archaea, Bacteria","2157, 2",NaN,Singlet


In [36]:
histone_description_summary = 'Face-to-Face (FtF) histones represent a distinct class of histones that have been identified in archaea (e.g., Haloferax volcanii) and select bacteria (e.g., Bdellovibrio bacteriovorus) using bioinformatic methods including AlphaFold2 structure prediction, sequence clustering (CLANS) and hidden Markov models (HMM) [schwab_histones_2024]. The structural configuration of FtF histones is comprised of a tetrameric assembly, wherein two distinct dimers engage via the interface of α2- and α3-helices, thereby establishing a ring-shaped structural arrangement. Multiple sequence alignment and structural analysis of FtF histones identified conserved positions, including the conserved RxTxxxD motif, that may be involved in DNA compactification [schwab_histones_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='FtF'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [37]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='FtF'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,FtF,variant_group,"Archaea, Bacteria","2157, 2",212,Singlet,212,Face-to-Face (FtF) histones represent a distin...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Nucleosomal

In [38]:
data_histone = {
    "id": "Nucleosomal",
    "level": 'variant_group',
    "taxonomic_span": 'Archaea, Bacteria',
    "taxonomic_span_id": '2157, 2',
    "description": None,
    "parent": 'Singlet',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [39]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Nucleosomal"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
196,Nucleosomal,variant_group,"Archaea, Bacteria","2157, 2",NaN,Singlet


In [40]:
histone_description_summary = 'Nucleosomal histones form structures similar to eukaryotic nucleosomes (including archaeal hypernucleosomes) by wrapping around DNA.'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Nucleosomal'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [41]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Nucleosomal'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Nucleosomal,variant_group,"Archaea, Bacteria","2157, 2",213,Singlet,213,Nucleosomal histones form structures similar t...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Bridge

In [42]:
data_histone = {
    "id": "Bridge",
    "level": 'variant_group',
    "taxonomic_span": 'Archaea, Bacteria, Viruses',
    "taxonomic_span_id": '2157, 2, 10239',
    "description": None,
    "parent": 'Singlet',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [43]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Bridge"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
1,Bridge,variant_group,"Archaea, Bacteria, Viruses","2157, 2, 10239",NaN,Singlet


In [44]:
histone_description_summary = 'DNA-bridge histones binds two strands of DNA together to form bridges.'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Bridge'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [45]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Bridge'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Bridge,variant_group,"Archaea, Bacteria, Viruses","2157, 2, 10239",214,Singlet,214,DNA-bridge histones binds two strands of DNA t...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Bacterial dimers

In [ ]:
data_histone = {
    "id": "Bacterial dimers",
    "level": 'variant_group',
    "taxonomic_span": 'Bacteria',
    "taxonomic_span_id": '2',
    "description": None,
    "parent": 'Singlet',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [49]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Bacterial dimers"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
1,Bacterial dimers,variant_group,Bacteria,2,NaN,Singlet


In [50]:
histone_description_summary = 'Bacterial dimers have been defined as a group of bacterial histones capable of binding to DNA in order to form nucleohistone filaments. These histones have the capacity to dimerise, yet do not form tetramers. For the bacterial singlet Bd0055, which is highly expressed, a crystallographic structure was obtained in which one monomer is covalently linked to its partner via double symmetry [noauthor_histones_2023]. This group of bacterial histones also includes histones predicted using AlphaFold2 and CLANS clustering, which are characterised by a shortened α3-helix and form stable dimers but do not form tetramers due to the absence of conserved residues, similar to Bd0055 [schwab_histones_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Bacterial dimers'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [51]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Bacterial dimers'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Bacterial dimers,variant_group,Bacteria,2,215,Singlet,215,Bacterial dimers have been defined as a group ...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add H1-like_(Viruses)

In [52]:
data_histone = {
    "id": "H1-like_(Viruses)",
    "level": 'variant_group',
    "taxonomic_span": 'Viruses',
    "taxonomic_span_id": '10239',
    "description": None,
    "parent": 'Singlet',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [53]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "H1-like_(Viruses)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
85,H1-like_(Viruses),variant_group,Viruses,10239,NaN,Singlet


In [54]:
histone_description_summary = 'Viral H1 histone is a putative homolog of the eukaryotic linker histone H1 encoded by the giant DNA virus Medusavirus medusae (NCLDV). Unlike classical H1, it is not involved in tri-nucleosome compactization, indicating a functional divergence from its eukaryotic counterparts [toner_characterization_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='H1-like_(Viruses)'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [55]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H1-like_(Viruses)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H1-like_(Viruses),variant_group,Viruses,10239,216,Singlet,216,Viral H1 histone is a putative homolog of the ...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Close connections

In [56]:
cursor.close()
conn.close()
tunnel.stop()